In [1]:
import sys
import traceback
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import dateutil as dtl
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import sys
from sqlalchemy import create_engine

In [2]:
def get_prices(ticker, start_date, end_date):
    '''
    
    Connects to the morningstar API and returns a pandas DataFrame of a desired stock for a specified time frame
    
    ticker = str, what stock do you wish to trade (using ticker symbol)
    start_date = str, self explanatory
    end_date = str, self explanatory
    
    '''
    
    try:
        start_date = str(dtl.parser.parse(start_date).date())
        end_date = str(dtl.parser.parse(end_date).date())
        start = dt.datetime.strptime(start_date, '%Y-%m-%d')
        end = dt.datetime.strptime(end_date, '%Y-%m-%d')
        f = web.DataReader(ticker, 'morningstar', start, end)
        df = pd.DataFrame(f)
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Error with the API.')
        print(err)
        print(tberr)
    
    try:
        df['EMA_20'] = ta.EMA(df.Close.values, 20)  # 20 day exponential moving average
        df['EMA_50'] = ta.EMA(df.Close.values, 50)  # 50 day exponential moving average
        #df['EMA_200'] = ta.EMA(df.Close.values, 200)  # 200 day exponential moving average
        #df['RSI'] = ta.RSI(df.Close.values, 14)  # 14 day relative strength index
        #df['CCI'] = ta.CCI(high=df.High.values, low=df.Low.values, close=df.Close.values, timeperiod=20)  # 20 day commodity channel index
        #df['Momentum'] = ta.MOM(df.Close.values, 10)  # 10 day rolling momentum
        #df['Stoch_fastk'], df['Stoch_fastd'] = ta.STOCHF(high=df.High.values, low=df.Low.values, close=df.Close.values, fastk_period=14, fastd_period=3)  # Stochastic fast indicator
        df['OBV'] = ta.OBV(np.asarray(df['Close']), df['Volume'])  # On Balance Volume
        df['MACD'], df['MACDsignal'], df['MACDhist'] = ta.MACD(df.Close.values)
        df['BBANDupper'], df['BBANDmiddle'], df['BBANDlower'] = ta.BBANDS(df['Close'], timeperiod=5)
        df['BBANDdiff'] = df['BBANDupper']-df['BBANDlower']
        df['BBANDdiff_norm'] = df.BBANDdiff/df['Close'].astype('float', inplace=True)
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Probably something wrong with TA-Lib')
        print(err)
        print(tberr)
        
    try:
        df['Buy'] = 0
        df['Sell'] = 0
        df['Stop Loss Sell'] = 0
        df['Sold at Sell Signal'] = 0
        df['Sold at 10% Profit'] = 0
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Hmm...')
        print(err)
        print(tberr)
    
    return df

In [3]:
def get_values():
    '''
    
    Connects to local SQL database and returns all information for the last 2 years as a pandas DataFrame
    
    '''    
    
    DSN_Name = 'traderDSN'
    Login_ID = 'basic'
    pwd = 'pwd1'
    
    try:
        # The engine does the same job as a connection and a cursor
        engine = create_engine(r'mssql+pyodbc://'+Login_ID+':'+pwd+'@'+ DSN_Name) 
        
        # Build query here:
        query = '''
        SELECT * FROM (\n\
        SELECT CAST([open] as DECIMAL(8,2)) as [open],\
        CAST([high] as DECIMAL(8,2)) as high,\
        CAST([low] as DECIMAL(8,2)) as low,\
        CAST([adjusted close] as DECIMAL(8,2)) as [adjusted close],\
        [volume], CAST([date] as DATE) as date,\
        [Name],\
        [Ticker]\
        FROM [algo].[stocks].[Adjusted_Fortune_500]\n)x\n\n\
        
        WHERE date > DATEADD(year, -2, (GETDATE()-1))
        '''
        
        # Importing a Pandas Dataframe: 
        df1 = pd.read_sql(query,engine)
        
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('I didn\'t write this bit.')
        print(err)
        print(tberr)
        
    
    try:
        df1['EMA_20'] = ta.EMA(df1['adjusted close'], 20)  # 20 day exponential moving average
        df1['EMA_50'] = ta.EMA(df1['adjusted close'], 50)  # 50 day exponential moving average
        df1['MACD'], df1['MACDsignal'], df1['MACDhist'] = ta.MACD(df1['adjusted close'])
        df1['BBANDupper'], df1['BBANDmiddle'], df1['BBANDlower'] = ta.BBANDS(df1['adjusted close'], timeperiod=5)
        df1['BBANDdiff'] = df1['BBANDupper']-df1['BBANDlower']
        df1['BBANDdiff_norm'] = df1.BBANDdiff/df1['adjusted close'].astype('float', inplace=True)
        
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Check TA-LIB')
        print(err)
        print(tberr)
        
    
    try:
        df1['Buy'] = 0
        df1['Sell'] = 0
        df1['Stop Loss Sell'] = 0
        df1['Sold at Sell Signal'] = 0
        df1['Sold at 10% Profit'] = 0
        df1['Stock Bought'] = ''
        df1['Stock Sold'] = ''
        
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('You\'re an idiot if something here is wrong')
        print(err)
        print(tberr)
    
    
    return df1

In [4]:
def volatile(df, no_of_stocks = 5):
    '''
    
    Function to calculate to relative volatility of stocks.
    Returns a list of stocks starting with the most historically volatile
    
    df = dataframe
    no_of_stocks = int, number of stocks to return in a list
    
    '''
    
    
    try:
        df.reset_index(inplace=True)
        vol_tickers = list(df.groupby('Ticker').mean().sort_values(by = 'BBANDdiff_norm', ascending = False).head(no_of_stocks).index)
        df.set_index(['Ticker', 'date'], inplace=True)
        
        return vol_tickers
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Something wrong with volatile()')
        print(err)
        print(tberr)

In [5]:
def trade_EMA_20_50(ticker, start_date, end_date, starting_fund, investment_per_trade):
    '''
    
    Trades on the standard moving average cross technique using the 20 and 50 day exponential moving averages.
    Returns the final funds after the specified time frame.
    
    ticker = str, what stock do you wish to trade (using ticker symbol)
    start_date = str, self explanatory
    end_date = str, self explanatory
    starting_fund = int, the initial amount of money, a minimum value of 10000 is suggested
    investment_per_trade = int, percentage of your fund you wish to invest per trade, a minimum value of 10% is suggested
    
    '''
    
    df = get_prices(ticker, start_date, end_date)
    
    wallet_ema = starting_fund
    risk_factor = investment_per_trade / 20
    stock_count_ema = 0
    stock_value_ema = 0
    
    bought = False
    bought_price = np.nan
    sold_for_profit = False
    
    try:    
        for index,row in df.iterrows():
            if np.isnan(row['EMA_50']) == False:
        
                # BUY STOCK AT UPWARDS CROSS ----------
                if all([row['EMA_50'] <= row['EMA_20'], bought == False, sold_for_profit == False, wallet_ema > 0]): 
                    amount = (wallet_ema * risk_factor) // row['Open']  # Buys max amount of whole stocks as is possible
                    wallet_ema = wallet_ema - (amount * row['Open'])
                    stock_count_ema = stock_count_ema + amount
                    stock_value_ema = stock_count_ema * row['Open']
                        
                    bought_price = row['Open']
                        
                    df.loc[index, 'Buy'] += 1
                    df.loc[index, 'Sell'] += 0
                    
                        
                    bought = True
                               
                # SELL STOCK AT 5% PROFIT ----------
                elif all([row['Close'] >= bought_price*1.1, bought == True, stock_count_ema > 0]):
                    wallet_ema = wallet_ema + (stock_count_ema*row['Close'])
                    stock_count_ema = 0
                    stock_value_ema = 0
                        
                    df.loc[index,'Buy'] += 0
                    df.loc[index,'Sell'] += 1
                    df.loc[index,'Sold at 10% Profit'] += 1
                    
                        
                    bought = False
                    sold_for_profit = True
                    bought_price = np.nan
                
                # RESET THE SOLD_FOR_PROFIT VARIABLE AT A DOWNWARDS CROSS ----------
                elif all([row['EMA_50'] > row['EMA_20'], bought == False]):                
                    sold_for_profit = False
                             
                # SELL STOCK AT DOWNWARDS CROSS ----------
                elif all([row['EMA_50'] > row['EMA_20'], bought == True, stock_count_ema > 0]):                
                    sold_for_profit = False
                    
                    wallet_ema = wallet_ema + (stock_count_ema*row['Close'])
                    stock_count_ema = 0
                    stock_value_ema = 0
                        
                    df.loc[index,'Buy'] += 0
                    df.loc[index,'Sell'] += 1
                    df.loc[index,'Sold at Sell Signal'] += 1
                        
                    bought = False
                    bought_price = np.nan
                                        
                # STOP LOSS (-5%) ----------
                elif all([row['Close'] < (bought_price*0.95), bought == True, stock_count_ema > 0]):                
                    sold_for_profit = False
                    
                    wallet_ema = wallet_ema + (stock_count_ema*row['Close'])
                    stock_count_ema = 0
                    stock_value_ema = 0
                        
                    df.loc[index,'Buy'] = 0
                    df.loc[index,'Sell'] += 1
                    df.loc[index,'Stop Loss Sell'] += 1
                        
                    bought = False
                    bought_price = np.nan
                                        
                # DO NOTHING ----------
                else:
                    df.loc[index,'Buy'] += 0
                    df.loc[index,'Sell'] += 0
                    
            else:
                pass

    
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Get debugging mate.')
        print(err)            
        print(tberr)
        
    ema_val = wallet_ema + stock_value_ema
    
    print('20 day / 50 day EMA')
    print('='*60)
    print('Stock                   : ', ticker)
   # print('Time-Period             : ', (df.shape[0])/261 , 'years')
    print('Initial funds           : ', starting_fund)
    print('Current funds           : ', wallet_ema)
    print('Realised profit         : ', wallet_ema - (starting_fund))
    print('Total asset value       : ', wallet_ema + stock_value_ema)
    #print('ROI                     : ', ((wallet_ema + stock_value)/(starting_fund / 2)),'%')
    print('Stock count             : ', stock_count_ema)
    print('Stock value             : ', stock_value_ema)
    print('No. of trades           : ', df['Buy'].sum())
    print('Sold at 10% profit      : ', df['Sold at 10% Profit'].sum())
    print('Sold at sell signal     : ', df['Sold at Sell Signal'].sum())
    print('Sold at stop loss       : ', df['Stop Loss Sell'].sum())
    print('-'*60,'\n')
    
    
    
    return ema_val

In [6]:
def trade_EMA_20_50_df(ticker, start_date, end_date, starting_fund, investment_per_trade):
    '''
    
    Trades on the standard moving average cross technique using the 20 and 50 day exponential moving averages.
    Returns a pandas DataFrame showing the dates a buy/sell signal was triggered, and which stock(s) were bought/sold.
    
    ticker = str, what stock do you wish to trade (using ticker symbol)
    start_date = str, self explanatory
    end_date = str, self explanatory
    starting_fund = int, the initial amount of money, a minimum value of 10000 is suggested
    investment_per_trade = int, percentage of your fund you wish to invest per trade, a minimum value of 10% is suggested
    
    '''
    
    df = get_prices(ticker, start_date, end_date)
    
    wallet_ema = starting_fund
    risk_factor = investment_per_trade / 20
    stock_count_ema = 0
    stock_value_ema = 0
    
    bought = False
    bought_price = np.nan
    sold_for_profit = False
    
    try:    
        for index,row in df.iterrows():
            if np.isnan(row['EMA_50']) == False:
        
                # BUY STOCK AT UPWARDS CROSS ----------
                if all([row['EMA_50'] <= row['EMA_20'], bought == False, sold_for_profit == False, wallet_ema > 0]): 
                    amount = (wallet_ema * risk_factor) // row['Open']  # Buys max amount of whole stocks as is possible
                    wallet_ema = wallet_ema - (amount * row['Open'])
                    stock_count_ema = stock_count_ema + amount
                    stock_value_ema = stock_count_ema * row['Open']
                        
                    bought_price = row['Open']
                        
                    df.loc[index, 'Buy'] += 1
                    df.loc[index, 'Sell'] += 0
                    df.loc[index, 'Stock Bought'] = str(ticker)
                        
                    bought = True
                               
                # SELL STOCK AT 5% PROFIT ----------
                elif all([row['Close'] >= bought_price*1.1, bought == True, stock_count_ema > 0]):
                    wallet_ema = wallet_ema + (stock_count_ema*row['Close'])
                    stock_count_ema = 0
                    stock_value_ema = 0
                        
                    df.loc[index,'Buy'] += 0
                    df.loc[index,'Sell'] += 1
                    df.loc[index,'Sold at 10% Profit'] += 1
                    df.loc[index, 'Stock Sold'] = ticker
                        
                    bought = False
                    sold_for_profit = True
                    bought_price = np.nan
                
                # RESET THE SOLD_FOR_PROFIT VARIABLE AT A DOWNWARDS CROSS ----------
                elif all([row['EMA_50'] > row['EMA_20'], bought == False]):                
                    sold_for_profit = False
                             
                # SELL STOCK AT DOWNWARDS CROSS ----------
                elif all([row['EMA_50'] > row['EMA_20'], bought == True, stock_count_ema > 0]):                
                    sold_for_profit = False
                    
                    wallet_ema = wallet_ema + (stock_count_ema*row['Close'])
                    stock_count_ema = 0
                    stock_value_ema = 0
                        
                    df.loc[index,'Buy'] += 0
                    df.loc[index,'Sell'] += 1
                    df.loc[index,'Sold at Sell Signal'] += 1
                    df.loc[index, 'Stock Sold'] = ticker
                        
                    bought = False
                    bought_price = np.nan
                                        
                # STOP LOSS (-5%) ----------
                elif all([row['Close'] < (bought_price*0.95), bought == True, stock_count_ema > 0]):                
                    sold_for_profit = False
                    
                    wallet_ema = wallet_ema + (stock_count_ema*row['Close'])
                    stock_count_ema = 0
                    stock_value_ema = 0
                        
                    df.loc[index,'Buy'] = 0
                    df.loc[index,'Sell'] += 1
                    df.loc[index,'Stop Loss Sell'] += 1
                    df.loc[index, 'Stock Sold'] = ticker
                        
                    bought = False
                    bought_price = np.nan
                                        
                # DO NOTHING ----------
                else:
                    df.loc[index,'Buy'] += 0
                    df.loc[index,'Sell'] += 0
                    
            else:
                pass

    
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Cheers for reading this far lozza!')
        print(err)            
        print(tberr)
    
    df = df.loc[:, 'Buy':]
    
    return df

In [7]:
def trade(ticker, start_date, end_date, starting_fund = 10000, in_play = 50, no_of_stocks = 5):
    '''
    
    ticker = str, what stock do you wish to trade (using ticker symbol)
    start_date = str, self explanatory
    end_date = str, self explanatory
    starting_fund = int, the initial amount of money, a minimum value of 10000 is suggested
    in_ply = int, the percentage of your starting_fund that you wish to have 'in play'
    no_of_stocks = int, number of stocks you wish to trade on
    
    '''
    
    
    
    investment_per_trade = in_play / (no_of_stocks-1)
    
    #in_play = in_play / 100    
    starting_fund = (starting_fund*in_play / 100) / (no_of_stocks-1)
    
    try:
        print('\n')
        ema_val = trade_EMA_20_50(ticker, start_date, end_date, starting_fund, investment_per_trade)
        
        total_val = ema_val
            
        print('Total value: ', total_val)
            
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Probably entered an argument incorrectly. Increase function usability.')
        print(err)            
        print(tberr)
        
    return total_val

In [8]:
def trade_df(ticker, start_date, end_date, starting_fund = 10000, in_play = 50, no_of_stocks = 5):
    '''
    
    ticker = str, what stock do you wish to trade (using ticker symbol)
    start_date = str, self explanatory
    end_date = str, self explanatory
    starting_fund = int, the initial amount of money, a minimum value of 10000 is suggested
    in_ply = int, the percentage of your starting_fund that you wish to have 'in play'
    no_of_stocks = int, number of stocks you wish to trade on
    
    '''
    
    investment_per_trade = in_play / (no_of_stocks-1)
    
    #in_play = in_play / 100    
    starting_fund = (starting_fund*in_play / 100) / (no_of_stocks-1)
    
    try:
        #print('\n')
        df = trade_EMA_20_50_df(ticker, start_date, end_date, starting_fund, investment_per_trade)
        
        #total_val = ema_val
            
        #print('Total value: ', total_val)
            
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Probably entered an argument incorrectly. Increase function usability.')
        print(err)            
        print(tberr)
        
    return df

In [9]:
def final_model(start_date, end_date, starting_fund, in_play = 100, no_of_stocks = 5):
    '''
    
    Combines all other funtions into one and writes results into a csv.
    
    start_date = str, self explanatory
    end_date = str, self explanatory
    starting_fund = int, the initial amount of money, a minimum value of 10000 is suggested
    in_ply = int, the percentage of your starting_fund that you wish to have 'in play'
    no_of_stocks = int, number of stocks you wish to trade on
    
    '''
    
    #start_date = str(dtl.parser.parse(start_date).date() - dt.timedelta(days=50))
    df = get_values()
    no_of_stocks += 1
    vol_tickers = volatile(df, no_of_stocks)

    try:
        vol_tickers.remove('BRKA')  # Only one row for BRKA in database
    except:
        pass
    
    print('You are trading on: \n')
    for ticker in vol_tickers:
        print(ticker)
        #start_date = str(dtl.parser.parse(start_date).date() + dt.timedelta(days=50))
    
    
    total_vals = []
    for ticker in vol_tickers:
        total_val = trade(ticker, start_date, end_date, starting_fund, in_play, no_of_stocks)
        total_vals.append(total_val)
     
    df = pd.DataFrame()
    for ticker in vol_tickers:
        #df = get_prices(ticker, start_date, end_date)
        df1 = trade_df(ticker, start_date, end_date, starting_fund, in_play, no_of_stocks)
        df = df.append(df1)
        #print(df)
        
    print('\n\n')
    print('Youre fund now stands at: ', sum(total_vals))
    print('Return on Investment    : ', round(((sum(total_vals)-starting_fund)/starting_fund)*100, 2),'%')
    print('\n\n')
    
    agga = {
    'Buy': 'sum',
    'Sell': 'sum',
    'Stop Loss Sell': 'sum', 
    'Sold at Sell Signal': 'sum',
    'Sold at 10% Profit': 'sum',
    'Stock Bought': 'unique',
    'Stock Sold': 'unique'
        }
    
    try:
        dff = df.groupby('Date').agg(agga)
        dff = dff.drop(dff[dff.Buy < 1].index & dff[dff.Sell < 1].index)
        y = []
        for i in dff['Stock Bought'].astype('str'):
            i = i[1:-1].split(' ')
            i = [j for j in i if j != 'nan']
            y.append(i)
        dff['Stock Bought'] = y
        y = []
        for i in dff['Stock Sold'].astype('str'):
            i = i[1:-1].split(' ')
            i = [j for j in i if j != 'nan']
            y.append(i)
        dff['Stock Sold'] = y
        
    except:
        err = sys.exc_info()[1]
        tberr = traceback.format_exc()
        print('Yeah sack this for a laugh...')
        print(err)            
        print(tberr)
    
    dff.to_csv('buysell.csv')